<a href="https://colab.research.google.com/github/Reem8484/Reem220/blob/main/Copy_of_Reem_hamed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [63]:
#My dataset is available in googledrive; so I am accessing my drive from colab
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
#Once this is executed, you will see your drive appearing on the left hand side

Mounted at /content/drive


In [61]:
# Define paths to the dataset
train_dir = '/content/drive/MyDrive/Muffin vs Chihuahua/train'  # Update with your actual path
test_dir = '/content/drive/MyDrive/Muffin vs Chihuahua/test'    # Update with your actual path

In [65]:
import os
print(len(os.listdir('/content/drive/MyDrive/Muffin vs Chihuahua/train/chihuahua')))
print(len(os.listdir('/content/drive/MyDrive/Muffin vs Chihuahua/train/muffin')))
print(len(os.listdir('/content/drive/MyDrive/Muffin vs Chihuahua/test/chihuahua')))
print(len(os.listdir('/content/drive/MyDrive/Muffin vs Chihuahua/test/muffin')))

400
408
518
500


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Create ImageDataGenerator for training set
train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2  # Split 20% of the images for validation
)

# Load and prepare training data
train_data = train_datagen.flow_from_directory(
    train_dir,
    target_size=(256,256),
    batch_size=32,
    class_mode='binary',  # 'binary' for binary classification (Muffin vs chihuahua)
    subset='training'  # Specify 'training' for the training set
)

# Create ImageDataGenerator for validation set
validation_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2  # Note: Using the same validation split as in the training set
)

# Load and prepare validation data
validation_data = validation_datagen.flow_from_directory(
    train_dir,
    target_size=(256,256),
    batch_size=32,
    class_mode='binary',
    subset='validation'  # Specify 'validation' for the validation set
)

Found 647 images belonging to 2 classes.
Found 161 images belonging to 2 classes.


In [ ]:
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense,Conv2D,MaxPooling2D,Flatten,BatchNormalization,UpSampling2D, Dropout

In [ ]:
# create CNN model - custom-made

model = Sequential()

model.add(Conv2D(32,kernel_size=(3,3),padding='valid',activation='relu',input_shape=(256,256,3)))  # 32 filters
model.add(MaxPooling2D(pool_size=(2,2),strides=2,padding='valid'))

model.add(Conv2D(64,kernel_size=(3,3),padding='valid',activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2),strides=2,padding='valid'))

model.add(Conv2D(128,kernel_size=(3,3),padding='valid',activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2),strides=2,padding='valid'))

model.add(Flatten())

model.add(Dense(128,activation='relu')) #feature reduction
model.add(Dense(64,activation='relu'))
model.add(Dense(1,activation='sigmoid'))  #output layer

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
from keras.optimizers import Adam
model.compile(optimizer=Adam(learning_rate=0.001),loss='binary_crossentropy',metrics=['accuracy']) #binary_crossentropy - binary classification

In [ ]:
history = model.fit(train_data, epochs=20, validation_data=validation_data)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
21/21 ━━━━━━━━━━━━━━━━━━━━ 136s 6s/step - accuracy: 0.5163 - loss: 0.9318 - val_accuracy: 0.8323 - val_loss: 0.6408
Epoch 2/20
21/21 ━━━━━━━━━━━━━━━━━━━━ 102s 5s/step - accuracy: 0.7791 - loss: 0.5643 - val_accuracy: 0.7267 - val_loss: 0.5128
Epoch 3/20
21/21 ━━━━━━━━━━━━━━━━━━━━ 93s 4s/step - accuracy: 0.7930 - loss: 0.4836 - val_accuracy: 0.7950 - val_loss: 0.4464
Epoch 4/20
21/21 ━━━━━━━━━━━━━━━━━━━━ 94s 5s/step - accuracy: 0.8606 - loss: 0.3328 - val_accuracy: 0.8571 - val_loss: 0.3723
Epoch 5/20
21/21 ━━━━━━━━━━━━━━━━━━━━ 141s 4s/step - accuracy: 0.8811 - loss: 0.2665 - val_accuracy: 0.8261 - val_loss: 0.3615
Epoch 6/20
21/21 ━━━━━━━━━━━━━━━━━━━━ 101s 5s/step - accuracy: 0.9209 - loss: 0.1957 - val_accuracy: 0.7950 - val_loss: 0.4783
Epoch 7/20
21/21 ━━━━━━━━━━━━━━━━━━━━ 92s 4s/step - accuracy: 0.9369 - loss: 0.1550 - val_accuracy: 0.8385 - val_loss: 0.4188
Epoch 8/20
21/21 ━━━━━━━━━━━━━━━━━━━━ 93s 4s/step - accuracy: 0.9696 - loss: 0.0738 - val_accuracy: 0.7764 - val_l

In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history['accuracy'],color='red',label='train')
plt.plot(history.history['val_accuracy'],color='blue',label='validation')
plt.legend()
plt.show()

In [ ]:
test_datagen = ImageDataGenerator(rescale=1./255)
test_data = test_datagen.flow_from_directory(
    test_dir,
    target_size=(256,256),
    batch_size=32,
    class_mode='binary'
)

Found 1018 images belonging to 2 classes.


In [ ]:
#predict the test data
predictions = model.predict(test_data)

32/32 ━━━━━━━━━━━━━━━━━━━━ 138s 4s/step


In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

# Assuming you have ground truth labels (true_labels) and predicted labels (predictions)
true_labels = test_data.classes
predicted_labels = (predictions > 0.5).astype(int)  # Adjust the threshold as needed

# Calculate confusion matrix
cm = confusion_matrix(true_labels, predicted_labels)

# Print confusion matrix
print("Confusion Matrix:")
print(cm)

# Print classification report
print("Classification Report:")
print(classification_report(true_labels, predicted_labels))

Confusion Matrix:
[[261 257]
 [253 247]]
Classification Report:
              precision    recall  f1-score   support

           0       0.51      0.50      0.51       518
           1       0.49      0.49      0.49       500

    accuracy                           0.50      1018
   macro avg       0.50      0.50      0.50      1018
weighted avg       0.50      0.50      0.50      1018



In [ ]:
# create CNN model

model = Sequential()

model.add(Conv2D(32,kernel_size=(3,3),padding='valid',activation='relu',input_shape=(256,256,3)))  # 32 filters
model.add(BatchNormalization())  # added to reduce overfitting
model.add(MaxPooling2D(pool_size=(2,2),strides=2,padding='valid'))

model.add(Conv2D(64,kernel_size=(3,3),padding='valid',activation='relu'))
model.add(BatchNormalization())  # added to reduce overfitting
model.add(MaxPooling2D(pool_size=(2,2),strides=2,padding='valid'))

model.add(Conv2D(128,kernel_size=(3,3),padding='valid',activation='relu'))
model.add(BatchNormalization())  # added to reduce overfitting
model.add(MaxPooling2D(pool_size=(2,2),strides=2,padding='valid'))

model.add(Flatten())

model.add(Dense(128,activation='relu')) #feature reduction
model.add(Dropout(0.1))  # added to reduce overfitting
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.1))  # added to reduce overfitting
model.add(Dense(1,activation='sigmoid'))  #output layer

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
from keras.optimizers import Adam
model.compile(optimizer=Adam(learning_rate=0.001),loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
history = model.fit(train_data, epochs=20, validation_data=validation_data)

Epoch 1/20
21/21 ━━━━━━━━━━━━━━━━━━━━ 164s 8s/step - accuracy: 0.6647 - loss: 5.3529 - val_accuracy: 0.4907 - val_loss: 3.8757
Epoch 2/20
21/21 ━━━━━━━━━━━━━━━━━━━━ 158s 8s/step - accuracy: 0.7991 - loss: 3.2869 - val_accuracy: 0.6522 - val_loss: 1.5695
Epoch 3/20
21/21 ━━━━━━━━━━━━━━━━━━━━ 147s 7s/step - accuracy: 0.8462 - loss: 1.7187 - val_accuracy: 0.4969 - val_loss: 13.5058
Epoch 4/20
21/21 ━━━━━━━━━━━━━━━━━━━━ 147s 7s/step - accuracy: 0.8776 - loss: 1.6305 - val_accuracy: 0.4969 - val_loss: 17.0957
Epoch 5/20
21/21 ━━━━━━━━━━━━━━━━━━━━ 147s 7s/step - accuracy: 0.9311 - loss: 1.1592 - val_accuracy: 0.4969 - val_loss: 17.4074
Epoch 6/20
21/21 ━━━━━━━━━━━━━━━━━━━━ 152s 7s/step - accuracy: 0.9190 - loss: 1.1466 - val_accuracy: 0.4969 - val_loss: 21.7526
Epoch 7/20
21/21 ━━━━━━━━━━━━━━━━━━━━ 145s 7s/step - accuracy: 0.9258 - loss: 0.7122 - val_accuracy: 0.4969 - val_loss: 42.0134
Epoch 8/20
21/21 ━━━━━━━━━━━━━━━━━━━━ 145s 7s/step - accuracy: 0.9368 - loss: 0.4729 - val_accuracy: 0.496

In [ ]:
test_data = test_datagen.flow_from_directory(
    test_dir,
    target_size=(256,256),
    batch_size=32,
    class_mode='binary'
)

Found 1018 images belonging to 2 classes.


In [ ]:
predictions = model.predict(test_data)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


32/32 ━━━━━━━━━━━━━━━━━━━━ 49s 2s/step


In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

# Assuming you have ground truth labels (true_labels) and predicted labels (predictions)
true_labels = test_data.classes
predicted_labels = (predictions > 0.5).astype(int)  # Adjust the threshold as needed

# Calculate confusion matrix
cm = confusion_matrix(true_labels, predicted_labels)

# Print confusion matrix
print("Confusion Matrix:")
print(cm)

# Print classification report
print("Classification Report after applying techniques to handle overfitting:")
print(classification_report(true_labels, predicted_labels))

Confusion Matrix:
[[452  66]
 [432  68]]
Classification Report after applying techniques to handle overfitting:
              precision    recall  f1-score   support

           0       0.51      0.87      0.64       518
           1       0.51      0.14      0.21       500

    accuracy                           0.51      1018
   macro avg       0.51      0.50      0.43      1018
weighted avg       0.51      0.51      0.43      1018



In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense,Conv2D,MaxPooling2D,Flatten

resnet_model = Sequential()    #ResNet50 is a pre-trained model
pretrained_model = tf.keras.applications.ResNet50(include_top = False, #because i have my own image of a diff dimension
                                                input_shape = (256,256,3),
                                                pooling = 'max', classes = 2,
                                                weights = 'imagenet')
for layer in pretrained_model.layers:
    layer.trainable = False  #all layers in the Resnet50 is not trainable; keep those weights; I am going to add my own i/p and other layers

resnet_model.add(pretrained_model)
resnet_model.add(Flatten())
resnet_model.add(Dense(512, activation = 'relu'))
resnet_model.add(Dense(1, activation = 'sigmoid'))

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
from keras.optimizers import Adam
resnet_model.compile(optimizer=Adam(learning_rate=0.001),loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
history = resnet_model.fit(train_data, epochs=20, validation_data=validation_data)

Epoch 1/20
21/21 ━━━━━━━━━━━━━━━━━━━━ 186s 9s/step - accuracy: 0.4871 - loss: 4.0206 - val_accuracy: 0.8012 - val_loss: 0.5118
Epoch 2/20
21/21 ━━━━━━━━━━━━━━━━━━━━ 173s 8s/step - accuracy: 0.6243 - loss: 0.8966 - val_accuracy: 0.8447 - val_loss: 0.4038
Epoch 3/20
21/21 ━━━━━━━━━━━━━━━━━━━━ 182s 9s/step - accuracy: 0.7832 - loss: 0.5252 - val_accuracy: 0.8447 - val_loss: 0.4192
Epoch 4/20
21/21 ━━━━━━━━━━━━━━━━━━━━ 178s 9s/step - accuracy: 0.7934 - loss: 0.4796 - val_accuracy: 0.7267 - val_loss: 0.5234
Epoch 5/20
21/21 ━━━━━━━━━━━━━━━━━━━━ 174s 8s/step - accuracy: 0.7745 - loss: 0.4683 - val_accuracy: 0.8634 - val_loss: 0.3694
Epoch 6/20
21/21 ━━━━━━━━━━━━━━━━━━━━ 181s 9s/step - accuracy: 0.8233 - loss: 0.4303 - val_accuracy: 0.8696 - val_loss: 0.3566
Epoch 7/20
21/21 ━━━━━━━━━━━━━━━━━━━━ 173s 8s/step - accuracy: 0.7713 - loss: 0.5305 - val_accuracy: 0.8509 - val_loss: 0.3531
Epoch 8/20
21/21 ━━━━━━━━━━━━━━━━━━━━ 173s 8s/step - accuracy: 0.8242 - loss: 0.4014 - val_accuracy: 0.8509 - v

In [ ]:
predictions = resnet_model.predict(test_data)

32/32 ━━━━━━━━━━━━━━━━━━━━ 221s 7s/step


In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

# Assuming you have ground truth labels (true_labels) and predicted labels (predictions)
true_labels = test_data.classes
predicted_labels = (predictions > 0.5).astype(int)  # Adjust the threshold as needed

# Calculate confusion matrix
cm = confusion_matrix(true_labels, predicted_labels)

# Print confusion matrix
print("Confusion Matrix:")
print(cm)

# Print classification report
print("Classification Report after applying techniques to handle overfitting:")
print(classification_report(true_labels, predicted_labels))

Confusion Matrix:
[[263 255]
 [242 258]]
Classification Report after applying techniques to handle overfitting:
              precision    recall  f1-score   support

           0       0.52      0.51      0.51       518
           1       0.50      0.52      0.51       500

    accuracy                           0.51      1018
   macro avg       0.51      0.51      0.51      1018
weighted avg       0.51      0.51      0.51      1018

